In [1]:
import os
nb_path = os.path.abspath("")
from sys import path
from os.path import dirname

path.insert(0,  dirname(nb_path))

nb_path

'c:\\Users\\ethan\\coding_projects\\trilogy-public-models\\local_examples'

In [2]:
from preql import Dialects
from trilogy_public_models import models
env = models["bigquery.thelook_ecommerce"]

executor = Dialects.BIGQUERY.default_executor(environment=env)

In [3]:
QA_3 = """

key cancelled_orders <- filter orders.id where orders.status = 'Cancelled';
auto orders.id.cancelled_count <- count(cancelled_orders);

SELECT
    orders.users.city,
    orders.id.cancelled_count / orders.id.count -> cancellation_rate,
    orders.id.cancelled_count,
    orders.id.count,
    orders.created_at.year,
WHERE
    (orders.created_at.year = 2020)
    and orders.id.count>10
ORDER BY
    cancellation_rate desc;
"""

results = executor.execute_text(QA_3)


In [4]:
QA_3 = """
SELECT 
    order_items.status,
    count(order_items.status)->count
;
"""

results = executor.execute_text(QA_3)
for row in results[0]:
    print(row)


('Cancelled', 26989)
('Complete', 45480)
('Processing', 36446)
('Shipped', 53868)
('Returned', 18209)


In [5]:
QA_3 = """
property order_items.id.revenue <- order_items.sale_price*order_items.orders.item_count;
# shipped, complete, processing
auto order_items.bookable_revenue <- filter order_items.revenue where order_items.status IN ( 'Shipped');

rowset booked_rev <- SELECT
    order_items.orders.created_at.month,
    order_items.revenue,
    order_items.status
where order_items.status in ('Shipped');

SELECT 
    booked_rev.order_items.orders.created_at.month,
    sum(booked_rev.order_items.revenue) -> bookable_revenue,
ORDER BY 
    booked_rev.order_items.orders.created_at.month DESC;
"""

results = executor.execute_text(QA_3)


In [6]:
for row in results[0]:
    print(row)

(12, 473807.6303511411)
(11, 432260.82053734735)
(10, 424294.310325861)
(9, 411811.33041501045)
(8, 370991.02048921585)
(7, 361968.950483799)
(6, 358162.1802742481)
(5, 967106.9208650589)
(4, 628281.8108203411)
(3, 605247.5007376671)
(2, 510985.94054055214)
(1, 503019.36043047905)
